In [1]:
import geopandas as gpd
from geopandas.geoseries import *
import pandas as pd 

# Get shapes of Berlin's Planungsräume 
# 448 PLRs
# source: Amt für Statistik Berlin-Brandenburg
# https://fbinter.stadt-berlin.de/fb/berlin/service_intern.jsp?id=s_lor_plan@senstadt&type=WFS

file_path_of_plr = 'LOR_Planungsraeume_2019.geojson'
plr_geo = gpd.read_file(file_path_of_plr)


# Get additional data to Berlin's Planungsräume
# source see above (FIS-Broker)

plr_data = pd.read_csv('LOR_Planungsraeume_2019.csv', sep=';')
plr_data.rename(columns={'Planungsraum': 'PLANUNGSRAUM', 'Bezirksregion': 'BEZIRKSREGION'}, inplace=True)

plr = plr_geo.merge(plr_data, on=['PLANUNGSRAUM', 'BEZIRKSREGION'])
plr.drop(['DATUM_GUELTIG_AB', 'Flächengröße in m²'], axis=1, inplace=True)
plr.rename(columns={'Schlüssel': 'SCHLUESSEL'}, inplace=True)
plr['SCHLUESSEL'] = plr['SCHLUESSEL'].astype(str)

# plr_geo = plr
# plr_geo.drop(['BEZIRKSNAME', 'BEZIRKSREGION', 'FLAECHENGROESSE_IN_M2', 'PROGNOSERAUM'], axis=1, inplace=True)
# plr_geo.rename(columns={'PLANUNGSRAUM': 'PLR_NAME'}, inplace=True)
# plr_geo.to_file('output_berlin-lor.planungsraeume.geojson', driver='GeoJSON')


# buffer and projection in meter
projection = "EPSG:5678" # https://epsg.io/5678
# buffer in meter (3 km/h walking pace; what is reachable in 10 min if your start at the boundary of PLR) => 500 m
bufferSize = 500 
plr = plr.to_crs(projection)

In [2]:
# 11522 forests
file_path_of_forests = 'waldflaechen.geojson'
forests = gpd.read_file(file_path_of_forests)
forests.insert(0, 'New_ID', range(0, 0 + len(forests)))
forests.to_file("output_waldflaechen.geojson", driver="GeoJSON")
#forests = forests.to_crs(projection)
forests["area_text"] = (forests["area_text"].replace(',', '.', regex=True).astype(float))

# # 433585749.9072622 => 43358.5749907 hectar Berlin and Brandenburg
# # forests['area_text'].sum()
# # 282036587.22155315 => 28203.658722155315 hectar in Berlin
# # forests[(forests.land == 'Berlin')]['area_text'].sum()
# # forests[(forests.land == 'Brandenburg')]['area_text'].sum() #151549162.68570915

plr['area'] = None
plr['bufferedGeometry'] = None
plr['bufferedArea'] = None
plr['bufferedArea'] = None

bufferedGeometry = plr.geometry.to_crs(projection)
copyGeometry = bufferedGeometry
bufferedGeometry = bufferedGeometry.buffer(bufferSize)

for index, row in plr.iterrows():
    plr.loc[index, 'area'] = copyGeometry[index].area / 10000
    plr.loc[index, 'bufferedArea'] = bufferedGeometry[index].area / 10000

copyGeometry = bufferedGeometry.to_crs("EPSG:4326")
plr['bufferedGeometry'] = bufferedGeometry

# add open space to calc which of them intersects with bufferedGeometry 
file_path_of_open_spaces = 'gruenflaechen.geojson'
open_spaces = gpd.read_file(file_path_of_open_spaces)

open_spaces = open_spaces.loc[open_spaces['KATASTERFL'] > 0]

open_spaces.KATASTERFL.sum()
open_spaces['AREA'] = None

open_spaces.insert(0, 'New_ID', range(len(forests), len(forests) + len(open_spaces)))
open_spaces.head()
open_spaces.to_file('output_gruenflaechen.geojson', driver='GeoJSON')

open_spaces = open_spaces.to_crs(projection)

for index, row in open_spaces.iterrows():
    open_spaces.loc[index, 'AREA'] = row.geometry.area
open_spaces.AREA.sum() / 10000

# data = pd.read_csv('data_PLZ.csv', sep=';')
# data = data.drop(columns="geometry")
# data = data.drop(columns="bufferedGeometry")
# data = data[['New_ID', 'SCHLUESSEL', 'PLR_NAME']]
# data['New_ID'] = data['New_ID'].astype(str)
# data['SCHLUESSEL'] = data['SCHLUESSEL'].astype(str)

# gdf = gpd.GeoDataFrame(data, geometry = copyGeometry)
# gdf.to_file('buffer.geojson', driver='GeoJSON')


# 2509 open spaces
# open_spaces = open_spaces.to_crs(projection)

plr['openSpacesInBufferedPLR'] = np.empty((len(plr), 0)).tolist()
plr['openSpacesInPLR'] = np.empty((len(plr), 0)).tolist()
plr['allOpenSpacesArea'] = np.empty((len(plr), 0)).tolist()
plr['allOpenSpacesBufferedArea'] = np.empty((len(plr), 0)).tolist()

# print(open_spaces['KATASTERFL'].sum()) # => 61376300.8 => 6137.63008 hectar

for indexPLR, rowPLR in plr.iterrows():
    for indexOS, rowOS in open_spaces.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(str(rowOS.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesBufferedArea'].append(rowOS.KATASTERFL)
        if rowPLR.geometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(str(rowOS.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesArea'].append(rowOS.KATASTERFL)
        if rowPLR.SCHLUESSEL.startswith('0'):
            plr.loc[indexPLR, 'SCHLUESSEL'] = rowPLR.SCHLUESSEL[1:]
    for indexF, rowF in forests.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowF.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(str(rowF.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesBufferedArea'].append(rowF.area_text)
        if rowPLR.geometry.intersects(rowF.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(str(rowF.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesArea'].append(rowF.area_text)

plr['bufferedGeometry'] = copyGeometry
newplr = plr

# sum of all PLRs => 89132.31009939541 hectar
sumOS = plr['area'].sum()
print(sumOS)

newplr.astype({'geometry': str}, {'bufferedGeometry': str}).to_csv('PLR_buffered.csv', sep=';')

89132.31009939541


In [3]:
import pandas as pd 
import numpy as np


newplr['numberOfSpacesInPLR'] = None
newplr['numberOfSpacesInBufferedPLR'] = None

for index, row in newplr.iterrows():
    newplr.loc[index, 'numberOfSpacesInPLR'] = len(row.openSpacesInPLR)
    newplr.loc[index, 'numberOfSpacesInBufferedPLR'] = len(row.openSpacesInBufferedPLR)

data = pd.read_csv('EWR201812E_Matrix.csv', sep=';')

data.rename(columns = {'RAUMID': 'SCHLUESSEL'}, inplace = True)
data = data[['SCHLUESSEL', 'E_E']]
data['E_E'] = data['E_E'].astype(str)

data['SCHLUESSEL'] = data['SCHLUESSEL'].astype(str)
newplr['SCHLUESSEL'] = newplr['SCHLUESSEL'].astype(str)

output = pd.merge(newplr, data, on='SCHLUESSEL', how='left')

output = output.rename(columns={'E_E': 'population'})

for index, row in output.iterrows():
    output.loc[index, 'allOpenSpacesBufferedArea'] = sum(row.allOpenSpacesBufferedArea) / 10000
    output.loc[index, 'allOpenSpacesArea'] = sum(row.allOpenSpacesArea) / 10000

# 3748148
# print(output['population'].sum())

output = output.to_crs('EPSG:4326')
#del output['bufferedGeometry']
output.count()
output.head()
#output.astype({'geometry': str}).to_csv('data.csv', index=False, header=True)

,BEZIRKSNAME,PLANUNGSRAUM,BEZIRKSREGION,PROGNOSERAUM,FLAECHENGROESSE_IN_M2,geometry,SCHLUESSEL,Bezirk,Prognoseraum,area,bufferedGeometry,bufferedArea,openSpacesInBufferedPLR,openSpacesInPLR,allOpenSpacesArea,allOpenSpacesBufferedArea,numberOfSpacesInPLR,numberOfSpacesInBufferedPLR,population
0,Mitte,Nördlicher Landwehrkanal,Tiergarten Süd,Zentrum,934609.72,"POLYGON ((13.35159 52.50976, 13.35180 52.50944...",1011105,Mitte,Zentrum,93.5224,"POLYGON ((13.34463 52.50831, 13.34450 52.50855...",421.314,"[13414, 13650, 13677, 13679, 13681, 13698, 137...","[13679, 13681, 13698, 13706, 13747, 13798, 138...",170.897,190.389,11,30,1421
1,Mitte,Wilhelmstraße,Regierungsviertel,Zentrum,634326.80,"POLYGON ((13.37678 52.51255, 13.37681 52.51246...",1011201,Mitte,Zentrum,63.48,"POLYGON ((13.36920 52.51128, 13.36921 52.51134...",312.579,"[13294, 13388, 13414, 13647, 13649, 13650, 136...","[13698, 13828]",165.861,188.05,2,29,2405
2,Mitte,Unter den Linden Nord,Regierungsviertel,Zentrum,744742.10,"POLYGON ((13.37673 52.51603, 13.37786 52.51611...",1011202,Mitte,Zentrum,74.5257,"POLYGON ((13.36953 52.51751, 13.36948 52.51800...",364.318,"[13647, 13649, 13650, 13667, 13668, 13669, 136...","[13649, 13695, 13697, 13704, 13766, 13769, 140...",4.64943,203.841,8,36,632
3,Mitte,Unter den Linden Süd,Regierungsviertel,Zentrum,548158.88,"POLYGON ((13.38109 52.51622, 13.38160 52.51526...",1011203,Mitte,Zentrum,54.8328,"POLYGON ((13.37858 52.51100, 13.37831 52.51108...",324.835,"[13647, 13649, 13667, 13669, 13671, 13674, 136...","[13709, 14024]",0.4761,192.413,2,32,1115
4,Mitte,Leipziger Straße,Regierungsviertel,Zentrum,763845.47,"POLYGON ((13.38679 52.51184, 13.38690 52.51115...",1011204,Mitte,Zentrum,76.4631,"POLYGON ((13.38555 52.50428, 13.38554 52.50429...",362.886,"[13294, 13309, 13388, 13657, 13658, 13659, 136...","[13659, 13674, 13686, 13696, 13768, 13787, 14007]",1.86165,25.9674,7,38,8279


In [4]:
import geopandas as gpd
from geopandas.geoseries import *

data = pd.read_csv("gruenflaechen.csv", sep=";").replace('-', '0', regex=True)
data["Größe in m² (Kataster)"] = pd.to_numeric(data["Größe in m² (Kataster)"])

data = data.loc[data['Größe in m² (Kataster)'] > 0.0]
data["Größe in m² (Kataster)"].sum() / 10000

#6147.55966

#len(data) #2525 2518

6147.55966

In [9]:
import geopandas as gpd

data = gpd.read_file('gruenflaechen.geojson')
data.head()
len(data) #2525
data = data.loc[data['KATASTERFL'] > 0]
data.count() # 2518
# data['KATASTERFL'].sum() #61475596.6

KENNZEICH     2518
BEZIRKNAME    2518
ORTSTLNAME    2514
OBJARTNAME    2518
NAMENR        2518
NAMEZUSATZ    1368
BAUJAHR        292
SANIERJAHR      98
KATASTERFL    2518
WIDMUNG       2400
PLANNR        2517
PLANNAME      2516
ANRECH_GF     2507
geometry      2518
dtype: int64

In [6]:
# # Get shapes of Berlin's Planungsräume 
# # 448 PLRs
# # source: Amt für Statistik Berlin-Brandenburg
# # https://fbinter.stadt-berlin.de/fb/berlin/service_intern.jsp?id=s_lor_plan@senstadt&type=WFS

# file_path_of_plr = 'LOR_Planungsraeume_2019.geojson'
# plr_geo = gpd.read_file(file_path_of_plr)


# # Get additional data to Berlin's Planungsräume
# # source see above (FIS-Broker)

# plr_data = pd.read_csv('data_PLZ.csv', sep=';')
# plr_data.rename(columns={'PLR_NAME': 'PLANUNGSRAUM', 'BZRNAME': 'BEZIRKSREGION'}, inplace=True)


# plr = plr_geo.merge(plr_data, on=['PLANUNGSRAUM', 'BEZIRKSREGION'])
# plr_geo = plr[['New_ID', 'SCHLUESSEL', 'PLANUNGSRAUM', 'geometry_x']]

# plr_geo.rename(columns={'PLANUNGSRAUM': 'PLR_NAME', 'geometry_x': 'geometry'}, inplace=True)

# plr_geo = gpd.GeoDataFrame(plr_geo, geometry=plr_geo.geometry)
# plr_geo.to_file('output_berlin-lor.planungsraeume.geojson', driver='GeoJSON')
